# DATA PRE-PROCESSING

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

In [2]:
dataset = pd.read_csv("./Datasets/LT.csv")
x = dataset.iloc[1:, 1:5].values
y = dataset.iloc[1:, -1].values

In [3]:
from sklearn.impute import SimpleImputer
imputer = SimpleImputer(missing_values=np.nan, strategy="mean")
imputer.fit(x[:, :-1])
x[:, :-1] = imputer.transform(x[:, :-1])

In [4]:
from sklearn.model_selection import train_test_split
xtrain, xtest, ytrain, ytest = train_test_split(x,y,test_size=0.2)

# LINEAR REGRESSION MULTIVARIABLE


In [5]:
from sklearn.linear_model import LinearRegression
lr_m =  LinearRegression()
lr_m.fit(xtrain,ytrain)
lr_m_pred = lr_m.predict(xtest)

In [6]:
np.set_printoptions(precision=2)
print(np.concatenate((ytest.reshape(len(ytest),1), lr_m.predict(xtest).reshape(len(lr_m.predict(xtest)),1)),1))

[[1358.65 1366.17]
 [1216.4  1219.47]
 [1679.8  1685.56]
 ...
 [2248.85 2231.09]
 [1564.45 1553.64]
 [1787.6  1776.79]]


# LINEAR REGRESSION SINGLE VARIABLE

In [7]:
from sklearn.linear_model import LinearRegression
lr = LinearRegression()
lr.fit(xtrain[:,0].reshape(-1,1), ytrain)

LinearRegression()

In [8]:
lr_pred = lr.predict(xtest[:,0].reshape(-1,1))
print(np.concatenate((ytest.reshape(-1,1), lr_pred.reshape(-1,1)),-1))

[[1358.65 1376.73]
 [1216.4  1236.68]
 [1679.8  1691.65]
 ...
 [2248.85 2206.87]
 [1564.45 1551.49]
 [1787.6  1768.03]]


# POLYNOMIAL LINEAR REGRESSION

In [9]:
from sklearn.preprocessing import PolynomialFeatures
poly = PolynomialFeatures(degree = 1)
X_poly = poly.fit_transform(x) 
lin2 = LinearRegression()

In [10]:
plrxtrain, plrxtest, plrytrain, plrytest = train_test_split(X_poly,y,test_size=0.2)

In [11]:
lin2.fit(plrxtrain, plrytrain)

LinearRegression()

In [12]:
plr_pred = lin2.predict(plrxtest)
print(np.concatenate((plrytest.reshape(-1,1), plr_pred.reshape(-1,1)),-1))

[[1310.8  1310.03]
 [1704.9  1694.88]
 [ 799.7   824.93]
 ...
 [1653.8  1658.02]
 [1614.05 1592.38]
 [1329.55 1322.84]]


# SUPPORT VECTOR REGRESSION

In [13]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
scaler.fit(x)

StandardScaler()

In [14]:
temp = scaler.fit_transform(x)
svrxtrain, svrxtest, svrytrain, svrytest = train_test_split(temp,y,test_size=0.2)

In [15]:
from sklearn.svm import SVR
clf_svr= SVR(kernel="rbf", C=100, gamma="auto")
clf_svr.fit(svrxtrain,svrytrain)

SVR(C=100, gamma='auto')

In [16]:
svr_pred = clf_svr.predict(svrxtest)
print(np.concatenate((svrytest.reshape(-1,1), svr_pred.reshape(-1,1)),-1))

[[ 609.9   668.77]
 [1424.9  1438.31]
 [2468.7  2504.37]
 ...
 [1631.1  1627.25]
 [1308.05 1326.97]
 [1665.45 1651.07]]


# DECISION TREE REGRESSION

# RANDOM FOREST REGRESSION

In [39]:
from sklearn.ensemble import RandomForestRegressor
rand_reg = RandomForestRegressor()
rand_reg.fit(xtrain, ytrain)
rand_pred = rand_reg.predict(xtest)

In [41]:
print(np.concatenate((ytest.reshape(-1,1), rand_pred.reshape(-1,1)),-1))

[[1358.65 1366.09]
 [1216.4  1217.13]
 [1679.8  1683.22]
 ...
 [2248.85 2233.82]
 [1564.45 1555.63]
 [1787.6  1775.53]]


# Comparing Model R2 score

Linear Regression Multivariable

Linear Regression Simple

Linear Regression Polynomial

Support Vector Regression

Decision Tree Regression

Random Forest Regression

# Results